In [6]:
from Functions_cg import *
import pandas as pd
import datetime as dt
from netCDF4 import Dataset
import scipy as scp
from dateutil.relativedelta import relativedelta
import matplotlib.pyplot as plt
import numpy as np
import scipy as scp
from cartopy import crs
import cartopy
import matplotlib.ticker as mticker
import matplotlib.ticker as ticker
import matplotlib.colors as mcolors
import os
from scipy.signal import tukey
import re
import multiprocessing
from functools import partial
import subprocess



In [ ]:
name = 'ERA5'
path_data = '/scratch/negishi/yan481/ERA5/'
path_ncr = '/apps/spack/negishi/apps/nco/4.6.7-gcc-4.8.5-gsgvpvo/bin/ncrcat'


path_outputs = f'/scratch/negishi/yan481/{name}/'
path_figures = f'/home/yan481/Cg/{name}/'
seasons = True
variable = 'v300'
nc_name = f'{variable}_{name}_6h_1979_1999.nc'
L0 = 20 #Longitude
E0 = 15 #[m/s] Threshold 
lambda1 = 2000  # km
lambda2 = 10000  # km
lambdaE = 4000 # km

# ncfile = Dataset(f'{path_outputs}{nc_name}')
ncfile = Dataset(f'f'/scratch/bell/castanev/{'ERA5'}/f'{'v300'}_{'ERA5'}_6h_1979_1999.nc')
lats = np.array(ncfile['lat'])
pos_NH = np.where(lats>=20)[0]
lats_NH = lats[lats>=20]
lons = np.array(ncfile['lon'])
time = np.array(ncfile['time'])[:]
dates_d = [dt.datetime.strptime(str(int(i)), "%Y%m%d%H") for i in time]

In [ ]:
if seasons == True:
    df = pd.DataFrame(index=dates_d, data=np.reshape(np.array(ncfile[variable])[:,pos_NH,:], [len(dates_d), lats_NH.shape[0] * lons.shape[0]])) 

    def calculate_day_climatology(tuple, time_col='time', dayofyear_col='dayofyear'): 
        n = tuple[0]; days = tuple[1]
        day_df1 = df[df[dayofyear_col].isin(days)]
        day_climatology = day_df1.groupby([dayofyear_col, time_col]).transform('mean')
        day_climatology.index = day_df1.index
        # anoma = day_df1 - day_climatology
        for i in np.unique(day_climatology.index.year):
            day_climatology_i = day_climatology[day_climatology.index.year == i]
            save_nc_3d(f'{path_outputs}temporal/clima_test/{i}{n}.nc', np.round(np.array(day_climatology_i),3).reshape(day_climatology_i.shape[0], lats_NH.shape[0], lons.shape[0]), lats_NH, lons, np.array([ii.strftime("%Y%m%d%H") for ii in day_climatology_i.index]), 'v300')


    unique_days = np.arange(1,367,1)
    intervals = [sublist.tolist() for sublist in np.array_split(unique_days, 15)]
    time_col='time'; df[time_col] = df.index.time
    dayofyear_col='dayofyear'; df[dayofyear_col] = df.index.dayofyear
    
    if __name__ == "__main__":
        args_list = [(n, days) for n, days in enumerate(intervals)]
        with multiprocessing.Pool(processes=150) as pool:
            pool.map(calculate_day_climatology, args_list)


    dates_d_index = pd.DatetimeIndex(dates_d)
    def calculate_anoma(ncfile_i):
        time_i = np.array(Dataset(f'{path_outputs}temporal/clima/clima{str(ncfile_i)}.nc')['time']); time_i = np.array([int(i) for i in time_i])
        clima_i = np.array(Dataset(f'{path_outputs}temporal/clima/clima{str(ncfile_i)}.nc')[variable])  
        for y in np.unique(dates_d_index[np.where(dates_d_index.dayofyear.isin(time_i))[0]].year):
            pos = np.where((dates_d_index.year==y)&(dates_d_index.dayofyear.isin(time_i)))[0]
            var_i = np.array(ncfile[variable])[pos,:,:]; var_i = var_i[:,pos_NH,:]
            anoma_i = var_i - clima_i[:len(pos)]
            save_nc_3d(f'{path_outputs}temporal/anoma/{str(y)}{str(ncfile_i)}.nc', anoma_i, lats_NH, lons, time[pos], 'v300_anom')


    list_data = os.listdir(f'{path_outputs}temporal/clima/')
    list_data = np.sort(np.array([re.findall(r'\d+', list_data[i])[0] for i in range(len(list_data))]).astype(int))
    for ncfile_i in list_data:
        calculate_anoma(ncfile_i)


    def filter_chunck(ncfile_i):
        df_anom = np.array(Dataset(f'{path_outputs}temporal/anoma/{str(ncfile_i)}.nc')['v300_anom'])
        time_i = np.array(Dataset(f'{path_outputs}temporal/anoma/{ncfile_i}.nc')['time'])
        series_t_filter = np.zeros([len(time_i), len(lats_NH), len(lons)]) * np.nan
        for t in range(len(time_i)):
            for num, lat in enumerate(lats_NH):
                serie = df_anom[t, num, :]
                km_1degree = np.pi*2*6371*np.cos(np.radians(lat))/360 # [km]
                k1 = 360*km_1degree/lambda1
                k2 = 360*km_1degree/lambda2
                series_t_filter[t,num,:] = fourier_filter_window(serie_i=serie, lons = lons, km_1degree = km_1degree, k1 = k1, k2 = k2)
        save_nc_3d(f'{path_outputs}temporal/anoma_filter/{str(ncfile_i)}.nc', series_t_filter, lats_NH, lons, time_i, 'v300_anom')


    list_data = os.listdir(f'{path_outputs}temporal/anoma/')
    list_data = np.sort(np.array([re.findall(r'\d+', list_data[i])[0] for i in range(len(list_data))]).astype(int))
    for sublist_data in [sublist.tolist() for sublist in np.array_split(list_data, 60)]:
        if __name__ == "__main__":
            with multiprocessing.Pool(processes=120) as pool:
                pool.map(filter_chunck, sublist_data)




    def calculate_envelope(ncfile_i):
        df_anom = np.array(Dataset(f'{path_outputs}temporal/anoma_filter/{str(ncfile_i)}.nc')['v300_anom'])
        time_i = np.array(Dataset(f'{path_outputs}temporal/anoma_filter/{str(ncfile_i)}.nc')['time'])
        E_thres = np.zeros_like(df_anom)*np.nan
        E_prim = np.zeros_like(df_anom)*np.nan
        for t in range(df_anom.shape[0]):
            for num, lat in enumerate(lats_NH):
                km_1degree = np.pi*2*6371*np.cos(np.radians(lat))/360 # [km]
                serie = df_anom[t,num,:]
                analytic_signal_filt = hilbert(serie)
                Envelope = np.abs(analytic_signal_filt)
                
                freq  = np.fft.fftfreq(len(Envelope), 1) 
                wavelengths = (1/freq)*np.abs(lons[1]-lons[0])*km_1degree #[km]  
                fourier = np.fft.fft(Envelope)/len(Envelope)
                filter = np.where((abs(wavelengths) < 4500))[0] 
                fourier_filtered = np.copy(fourier)
                fourier_filtered[filter] = 0
                Envelope_filter = np.fft.ifft(fourier_filtered* len(serie))
                
                
                # Cg ==========================================================================================
                E_threshold = filter_threshold_lenght(Envelope_filter, lons, E0, L0)
                E_thres[t,num,:] = E_threshold
                if E_threshold.any() == np.nan: print(f'More than 1 event {np.array(dates_d)[t]}')
                
                events_data = {}
                # Extract event data and store in the list
                current_event = []
                contt = 0
                
                for i, value in enumerate(E_threshold):
                    if value: current_event.append(value)          
                    else:
                        if current_event:
                            contt = contt+1
                            events_data[contt] = pd.Series(current_event, index = np.arange(i-len(current_event)+1,i+1,1))
                            current_event = []
                    if ((i+1 == len(E_threshold)) & (i+1 == len(current_event))):
                        contt = contt+1
                        events_data[contt] = pd.Series(current_event, index = np.arange(i-len(current_event)+1,i+1,1))
                        current_event = []
                
                for i in events_data.keys():
                    media_object = events_data[i].mean()
                    E_prim[t,num,events_data[i].index] = E_threshold[events_data[i].index] - media_object
                
                if (t in [400]) & (lat == 45.0):
                    fig = plt.figure(figsize=[10, 6])
                    ax = fig.add_subplot(1, 1, 1)
                    plt.plot(df_anom[t, num, :], c='k')
                    plt.plot(Envelope_filter, c='g')
                    plt.plot(E_prim[t, num, :], c='orange')
                    plt.savefig(path_figures + f'test{round(round(lat))}_{ncfile_i}_filtered_annual_clima6.png', dpi=500)

        save_nc_3d(f'{path_outputs}temporal/E_thres/{str(ncfile_i)}.nc', E_thres, lats_NH, lons, time_i, 'E')
        save_nc_3d(f'{path_outputs}temporal/E_prim/{str(ncfile_i)}.nc', E_prim, lats_NH, lons, time_i, 'E')


    list_data = os.listdir(f'{path_outputs}temporal/anoma_filter/')
    list_data = np.sort(np.array([re.findall(r'\d+', list_data[i])[0] for i in range(len(list_data))]).astype(int))
    for sublist_data in [sublist.tolist() for sublist in np.array_split(list_data, 60)]:
        if __name__ == "__main__":
            with multiprocessing.Pool(processes=120) as pool:
                pool.map(calculate_envelope, sublist_data)


    list_data = os.listdir(f'{path_outputs}temporal/E_thres/')
    list_data = np.sort(np.array([re.findall(r'\d+', list_data[i])[0] for i in range(len(list_data))]).astype(int))
    nco_command = f"module load nco && {path_ncr} {' '.join([f'{path_outputs}/temporal/E_thres/' + str(i) + '.nc' for i in list_data])} {os.path.join(path_outputs, 'temporal', f'E_thres_{name}_1979-1999.nc')}"
    result = subprocess.run(nco_command, shell=True)
    print(result.returncode)


    E_thres = np.array(Dataset(f'{path_outputs}temporal/E_thres_{name}_1979-1999.nc')['E'])
    E_seasonal_mean = np.empty([4,len(lats_NH),len(lons)]) * np.nan
    print(E_thres)
    print(E_thres.shape)
    time_i = np.array(Dataset(f'{path_outputs}temporal/E_thres_{name}_1979-1999.nc')['time'])
    dates_d_i = [dt.datetime.strptime(str(int(i)), "%Y%m%d%H") for i in time_i]
    for num, j, season in zip([0,1,2,3],[[12,1,2], [3,4,5], [6,7,8], [9,10,11]],['DJF','MAM','JJA','SON']):
        pos_season = np.where((pd.to_datetime(dates_d_i).month == j[0]) ^ (pd.to_datetime(dates_d_i).month == j[1]) ^ (pd.to_datetime(dates_d_i).month == j[2]))[0]
        E_seasonal_mean[num,:,:] = np.nanmean(E_thres[pos_season,:,:], axis=0)
    save_nc_3d(f'{path_outputs}temporal/E_seasonal_mean.nc', E_seasonal_mean, lats_NH, lons, np.arange(0,4,1).astype(str), 'E')


    E_seasonal_mean = np.array(Dataset(f'{path_outputs}temporal/E_seasonal_mean.nc')['E'])
    for num, season in zip([0,1,2,3],['DJF','MAM','JJA','SON']):
        vmin, vmax, colormap = colorm(10, 27, 15, 11, 'RdYlBu_r') 
        season_mean = E_seasonal_mean[num]
        map_orhographic(lons, lats_NH, vmin, vmax, season_mean,  colormap, f'{path_figures}E_{season}_{name}_ort_filterafter_annual_clima', units='[m/s]', topography = True)


In [ ]:
import scipy as sp

if seasons == True:

    lons_20_degrees = int(20/(lons[1]-lons[0]))
    
    def theta_prim(ncfile_i):
        E_prim = np.array(Dataset(f'{path_outputs}temporal/E_thres/{ncfile_i}.nc')['E'])
        time_i = np.array(Dataset(f'{path_outputs}temporal/E_thres/{ncfile_i}.nc')['time'])
        theta_prim_diff = np.zeros_like(E_prim) * np.nan 
        theta_prim = np.zeros_like(E_prim) * np.nan 

        for num, lat in enumerate(lats_NH):
            # print('------------------------')
            # print(num)
            print(lat)
            #events = {}
            for t in range(1,E_prim.shape[0]-1):
            # for t in range(1,10):
                serie = E_prim[t,num,:]
                if (np.count_nonzero(~np.isnan(serie)) == 0): continue


                not_nan_indices = np.diff(np.concatenate(([False], ~np.isnan(serie))))

                # Find the indices where the True values start and end
                indices = np.where(not_nan_indices == 1)[0]

                if len(indices) > 0:
                    if (len(serie) > indices[-1]) & (len(indices) % 2 > 0):
                        indices = np.append(indices, len(not_nan_indices))

                events = np.reshape(indices, [int(len(indices)/2), 2])

                for event_i in events:
                    # print(event_i)
                    if np.count_nonzero(~(serie[event_i[0]:event_i[1]] == np.zeros_like(serie[event_i[0]:event_i[1]]))) == 0: continue 
                    analytic_signal = hilbert(serie[event_i[0]:event_i[1]] - np.nanmean(serie[event_i[0]:event_i[1]]))
                    angles = np.round((np.unwrap((np.angle(analytic_signal))) + 4 * np.pi) % (4 * np.pi) - 2 * np.pi , 3)

                    angles[(angles<-3*np.pi/2) & (angles>=-2*np.pi)] = angles[(angles<-3*np.pi/2) & (angles>=-2*np.pi)] + 2*np.pi
                    angles[(angles>3*np.pi/2) & (angles<2*np.pi)] = angles[(angles>3*np.pi/2) & (angles<2*np.pi)] + 2*np.pi
                    theta_prim[t,num,event_i[0]:event_i[1]] = angles

                    len_event = event_i[1]-event_i[0]
                                    
                    serie_pre_cut = E_prim[t-1, num, max(0,event_i[0]-lons_20_degrees):min(len(serie),event_i[1]+lons_20_degrees)]
                    serie_post_cut = E_prim[t+1, num, max(0,event_i[0]-lons_20_degrees):min(len(serie),event_i[1]+lons_20_degrees)]
                        
                    serie_pre = max_subarray_of_length(serie_pre_cut, len_event) 
                    serie_post = max_subarray_of_length(serie_post_cut, len_event) 

                    if (np.count_nonzero(~np.isnan(serie_pre)) == 0) or (np.count_nonzero(~np.isnan(serie_post)) == 0): continue
                    else:
                        analytic_signal_pre = hilbert(serie_pre[~np.isnan(serie_pre)] - np.nanmean(serie_pre))
                        analytic_signal_post = hilbert(serie_post[~np.isnan(serie_post)] - np.nanmean(serie_post))

                        phase_pre = np.empty([len_event]) *np.nan
                        phase_post = np.empty([len_event]) *np.nan
                        angles_pre =  np.round((np.unwrap((np.angle(analytic_signal_pre)))+ 4 * np.pi) % (4 * np.pi) - 2 * np.pi , 3)
                        angles_post =  np.round((np.unwrap((np.angle(analytic_signal_post)))+ 4 * np.pi) % (4 * np.pi) - 2 * np.pi , 3)

                        angles_pre[(angles_pre<-3*np.pi/2) & (angles_pre>=-2*np.pi)] = angles_pre[(angles_pre<-3*np.pi/2) & (angles_pre>=-2*np.pi)] + 2*np.pi
                        angles_post[(angles_post>3*np.pi/2) & (angles_post<2*np.pi)] = angles_post[(angles_post>3*np.pi/2) & (angles_post<2*np.pi)] + 2*np.pi
                        phase_pre[int((len_event - len(analytic_signal_pre))/2) : int((len_event - len(analytic_signal_pre))/2) + len(analytic_signal_pre)] = angles_pre
                        phase_post[int((len_event - len(analytic_signal_post))/2) : int((len_event - len(analytic_signal_post))/2) + len(analytic_signal_post)] = angles_post

                        theta_prim_diff[t, num,event_i[0]:event_i[1]] = phase_post - phase_pre
                        

        save_nc_3d(f'{path_outputs}temporal/theta_prim_diff/{str(ncfile_i)}.nc', theta_prim_diff, lats_NH, lons, time_i, 'theta_prim')
        save_nc_3d(f'{path_outputs}temporal/theta_prim/{str(ncfile_i)}.nc', theta_prim, lats_NH, lons, time_i, 'theta_prim')

    list_data = os.listdir(f'{path_outputs}temporal/E_thres/')
    list_data = np.sort(np.array([re.findall(r'\d+', list_data[i])[0] for i in range(len(list_data))]).astype(int))
    for sublist_data in [sublist.tolist() for sublist in np.array_split(list_data, 70)]:
        if __name__ == "__main__":
            with multiprocessing.Pool(processes=160) as pool:
                pool.map(theta_prim, sublist_data)








    def w_prim(ncfile_i):
        print(ncfile_i)
        theta_prim = np.array(Dataset(f'{path_outputs}temporal/theta_prim_diff/{ncfile_i}.nc')['theta_prim'])
        time_i = np.array(Dataset(f'{path_outputs}temporal/theta_prim_diff/{ncfile_i}.nc')['time'])
        w_prim = np.zeros_like(theta_prim) * np.nan

          
        for t in range(1,len(time_i[1:-1])):
            w_prim_i = pd.DataFrame(theta_prim[t])

            w_prim_i[w_prim_i> np.pi] = (-w_prim_i[w_prim_i> np.pi] + 2*np.pi)*1000000 / (2*6*3600) # [rad/s]
            w_prim_i[w_prim_i< -np.pi] = (-w_prim_i[w_prim_i< -np.pi] - 2*np.pi)*1000000 / (2*6*3600) # [rad/s]
            w_prim_i[(w_prim_i<= np.pi) & (w_prim_i>= -np.pi)] = (-w_prim_i[(w_prim_i<= np.pi) & (w_prim_i>= -np.pi)])*1000000 / (2*6*3600) # [rad/s]

            w_prim[t,:,:] = w_prim_i
        save_nc_3d(f'{path_outputs}temporal/w_prim/{str(ncfile_i)}.nc', w_prim, lats_NH, lons, time_i, 'w_prim')

    list_data = os.listdir(f'{path_outputs}temporal/theta_prim_diff/')
    list_data = np.sort(np.array([re.findall(r'\d+', list_data[i])[0] for i in range(len(list_data))]).astype(int))
    for sublist_data in [sublist.tolist() for sublist in np.array_split(list_data,12)]:
        if __name__ == "__main__":
            with multiprocessing.Pool(processes=90) as pool:
                pool.map(w_prim, sublist_data) 

    

    a = 6371000 #[m]
    def k_prim(ncfile_i):
        print(ncfile_i)
        theta_prim = np.array(Dataset(f'{path_outputs}temporal/theta_prim/{ncfile_i}.nc')['theta_prim'])
        time_i = np.array(Dataset(f'{path_outputs}temporal/theta_prim/{ncfile_i}.nc')['time'])
        k_prim = np.zeros_like(theta_prim) * np.nan
        for num_lat in range(len(lats_NH)):
            print(lats_NH[num_lat])
            k_prim_i = pd.DataFrame(theta_prim[:,num_lat,:] - np.roll(theta_prim[:,num_lat,:], -2, axis=1)) #pre - post

            if (np.count_nonzero(~np.isnan(k_prim_i)) == 0): continue
            else:
                k_prim_i[k_prim_i> np.pi] = (-k_prim_i[k_prim_i> np.pi] + 2*np.pi)*1000000 / (2*abs(lons[1]-lons[0])*1*(np.pi/180) * a*np.cos(np.radians(lats_NH[num_lat])))  # [rad/m]
                k_prim_i[k_prim_i< -np.pi] = (-k_prim_i[k_prim_i< -np.pi] - 2*np.pi)*1000000 / (2*abs(lons[1]-lons[0])*1*(np.pi/180) * a*np.cos(np.radians(lats_NH[num_lat]))) # [rad/m]
                k_prim_i[(k_prim_i<= np.pi) & (k_prim_i>= -np.pi)] = (-k_prim_i[(k_prim_i<= np.pi) & (k_prim_i>= -np.pi)])*1000000 / (2*abs(lons[1]-lons[0])*1*(np.pi/180) * a*np.cos(np.radians(lats_NH[num_lat]))) # [rad/m]

                k_prim[:,num_lat,:] = np.roll(np.array(k_prim_i), 1, axis=1)

        k_prim[k_prim==0] = np.nan        
        save_nc_3d(f'{path_outputs}temporal/k_prim/{str(ncfile_i)}.nc', k_prim, lats_NH, lons, time_i, 'k_prim')

    list_data = os.listdir(f'{path_outputs}temporal/theta_prim/')
    list_data = np.sort(np.array([re.findall(r'\d+', list_data[i])[0] for i in range(len(list_data))]).astype(int))
    for sublist_data in [sublist.tolist() for sublist in np.array_split(list_data, 12)]:
        if __name__ == "__main__":
            with multiprocessing.Pool(processes=90) as pool:
                pool.map(k_prim, sublist_data)



    def group_vel(ncfile_i): 
        print(ncfile_i)
        w_prim = np.array(Dataset(f'{path_outputs}temporal/w_prim/{ncfile_i}.nc')['w_prim'])
        k_prim = np.array(Dataset(f'{path_outputs}temporal/k_prim/{ncfile_i}.nc')['k_prim'])
        time_i = np.array(Dataset(f'{path_outputs}temporal/k_prim/{ncfile_i}.nc')['time'])
        cg = w_prim / k_prim 

        save_nc_3d(f'{path_outputs}temporal/Cg/{str(ncfile_i)}.nc', cg, lats_NH, lons, time_i, 'Cg')

    list_data = os.listdir(f'{path_outputs}temporal/w_prim/')
    list_data = np.sort(np.array([re.findall(r'\d+', list_data[i])[0] for i in range(len(list_data))]).astype(int))
    for i in list_data:
        group_vel(i)
    

    list_data = os.listdir(f'{path_outputs}temporal/Cg/')
    list_data = np.sort(np.array([re.findall(r'\d+', list_data[i])[0] for i in range(len(list_data))]).astype(int))

    nco_command = f"module load nco && {path_ncr} {' '.join([f'{path_outputs}/temporal/Cg/' + str(i) + '.nc' for i in list_data])} {os.path.join(path_outputs, 'temporal', f'Cg_{name}_1979-1999.nc')}"
    result = subprocess.run(nco_command, shell=True)
    print(result.returncode)


    Cg = np.array(Dataset(f'{path_outputs}temporal/Cg_{name}_1979-1999.nc')['Cg'])
    # Cg[abs(Cg) > 100] = np.nan

    Cg_seasonal_mean = np.empty([4,len(lats_NH),len(lons)]) * np.nan
    time_i = np.array(Dataset(f'{path_outputs}temporal/Cg_{name}_1979-1999.nc')['time'])
    dates_d_i = [dt.datetime.strptime(str(int(i)), "%Y%m%d%H") for i in time_i]
    for num, j, season in zip([0,1,2,3],[[12,1,2], [3,4,5], [6,7,8], [9,10,11]],['DJF','MAM','JJA','SON']):
        pos_season = np.where((pd.to_datetime(dates_d_i).month == j[0]) ^ (pd.to_datetime(dates_d_i).month == j[1]) ^ (pd.to_datetime(dates_d_i).month == j[2]))[0]
        Cg_seasonal_mean[num,:,:] = np.nanmean(Cg[pos_season,:,:], axis=0)
    save_nc_3d(f'{path_outputs}temporal/Cg_seasonal_mean.nc', Cg_seasonal_mean, lats_NH, lons, np.arange(0,4,1).astype(str), 'Cg')


    Cg_seasonal_mean = np.array(Dataset(f'{path_outputs}temporal/Cg_seasonal_mean.nc')['Cg'])
    for num, season in zip([0,1,2,3],['DJF','MAM','JJA','SON']):
        vmin, vmax, colormap = colorm(10, 20, 15, 11, 'RdYlBu_r') 
        season_mean = Cg_seasonal_mean[num]
        
        map_orhographic(lons, lats_NH, vmin, vmax, season_mean,  colormap, f'{path_figures}Cg_{season}_{name}_ort_filterafter_annual_clima', units='[m/s]', topography = True)







In [7]:
import os
import netCDF4

directory_path = "/depot/wanglei/data/Reanalysis/ERA5/Cg/theta_prim/"

nc_files = [f for f in os.listdir(directory_path) if f.endswith('.nc')]

if nc_files:
    sample_file_path = os.path.join(directory_path, nc_files[0])
    with netCDF4.Dataset(sample_file_path, 'r') as nc_file:
        print(nc_file)
else:
    print("no .nc file in this dir.")


<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    dimensions(sizes): lat(281), lon(1440), time(100)
    variables(dimensions): float32 lat(lat), float32 lon(lon), <class 'str'> time(time), float32 theta_prim(time, lat, lon)
    groups: 


In [2]:
import netCDF4

# open NetCDF files
file_path = "/depot/wanglei/data/Reanalysis/ERA5/Cg/theta_prim/19790.nc"
dataset = netCDF4.Dataset(file_path, "r")

# get theta_prim value
theta_prim = dataset.variables['theta_prim'][:]

# print some theta_prim (e.g., first 100 theta value at (148. 333))
print(theta_prim[:1000, 148, 333])

# clsoe NetCDF files
dataset.close()


[   nan    nan    nan    nan    nan    nan    nan    nan    nan  2.701
  2.975  2.881    nan    nan    nan    nan    nan  2.673  2.027  1.123
  2.805    nan    nan    nan    nan    nan    nan    nan    nan    nan
    nan    nan    nan    nan    nan    nan    nan    nan    nan    nan
    nan    nan    nan    nan  2.375  2.33   2.875  2.245  2.209  1.48
  2.161  1.856    nan  2.966    nan  2.662  2.697  2.866  2.723  2.461
  2.749    nan    nan    nan    nan    nan  2.776  2.007  1.335  0.896
  0.48  -0.098 -0.574 -0.805 -0.701  0.601  0.867  1.949    nan    nan
    nan    nan    nan    nan    nan    nan    nan  2.76   1.797  0.758
  0.165 -2.462 -2.489    nan    nan    nan    nan    nan    nan    nan]


In [9]:
import netCDF4

# open NetCDF files
file_path = "/depot/wanglei/data/Reanalysis/ERA5/Cg/theta_prim_corrected/19790.nc"
dataset = netCDF4.Dataset(file_path, "r")

# get theta_prim value
theta_prim = dataset.variables['theta_prim'][:]

# print some theta_prim (e.g., first 100 theta value at (148. 333))
print(theta_prim[:1000, 148, 333])

# clsoe NetCDF files
dataset.close()

[   nan    nan    nan    nan    nan    nan    nan    nan    nan -2.132
  6.953  1.337    nan    nan    nan    nan    nan  4.483  4.404 -2.29
 -4.128    nan    nan    nan    nan    nan    nan    nan    nan    nan
    nan    nan    nan    nan    nan    nan    nan    nan    nan    nan
    nan    nan    nan    nan -1.047 -0.546 -0.738 -0.911 -1.164 -1.443
 -1.251 -1.54     nan -0.932    nan  0.671  0.672  3.776  3.183 -1.926
 -0.88     nan    nan    nan    nan    nan  5.802 -1.477 -1.932 -2.45
 -2.804 -3.481  2.572  2.468  2.814  3.975  4.294 -1.609    nan    nan
    nan    nan    nan    nan    nan    nan    nan -0.64  -1.943 -2.935
 -3.711 -4.828 -5.322    nan    nan    nan    nan    nan    nan    nan]


In [7]:
from netCDF4 import Dataset

# ncfile = Dataset(f'f'/scratch/bell/castanev/{'ERA5'}/f'{'v300'}_{'ERA5'}_6h_1979_1999.nc')
ncfile = Dataset(f'/scratch/negishi/yan481/KenYan_Traffic_Model/Interim_Slice_6hr/1979_1979_01_01_06_00_u.nc')
lats = np.array(ncfile['lat'])
pos_NH = np.where(lats>=20)[0]
lats_NH = lats[lats>=20]
lons = np.array(ncfile['lon'])
time = np.array(ncfile['time'])[:]

IndexError: lat not found in /

In [5]:
def save_nc_3d(path_name, var, lats, lons, dates_str, var_str):
    ncfile = Dataset(path_name, 'w')
    ncfile.createDimension('lat', len(lats))
    ncfile.createDimension('lon', len(lons))
    ncfile.createDimension('time', None)
    var_lats = ncfile.createVariable('lat', 'f', ('lat'))
    var_lons = ncfile.createVariable('lon', 'f', ('lon'))
    var_time = ncfile.createVariable('time', 'str', ('time'))
    var_lats[:] = lats
    var_lons[:] = lons
    var_time[:] = dates_str
    varr = ncfile.createVariable(var_str, 'f', ('time', 'lat', 'lon'), fill_value=None)
    varr[:, :, :] = var
    ncfile.close()

In [6]:
import numpy as np
from netCDF4 import Dataset
import os
from multiprocessing import Pool
from tqdm import tqdm

def calculate_wrapped_derivative(prev_val, curr_val, next_val, delta):
    diff = next_val - prev_val

    if diff > np.pi:
        return (prev_val + 2 * np.pi - next_val) / (2 * delta)
    elif diff < -np.pi:
        return (prev_val - 2 * np.pi - next_val) / (2 * delta)
    else:
        return (prev_val - next_val) / (2 * delta)


def w_prim(ncfile_i):
    print(ncfile_i)
    theta_prim = np.array(Dataset(f'/depot/wanglei/data/Reanalysis/ERA5/Cg/theta_prim/{ncfile_i}')['theta_prim'])
    time_i = np.array(Dataset(f'/depot/wanglei/data/Reanalysis/ERA5/Cg/theta_prim/{ncfile_i}')['time'])
    w_prim = np.zeros_like(theta_prim) * np.nan

    delta_t = 6 * 3600  # assuming delta_t is in seconds

    for t in range(1, len(time_i[1:-1])):
        for j in range(theta_prim.shape[1]):
            for k in range(theta_prim.shape[2]):
                w_prim[t, j, k] = -calculate_wrapped_derivative(
                    theta_prim[t-1, j, k], 
                    theta_prim[t, j, k], 
                    theta_prim[t+1, j, k], 
                    delta_t
                )
    save_nc_3d(f'/scratch/negishi/yan481/KenYan_Traffic_Model/Valentina Cg Calculation/{str(ncfile_i)}.nc', w_prim, lats_NH, lons, time_i, 'w_prim')


if __name__ == "__main__":
    path_to_data = "/depot/wanglei/data/Reanalysis/ERA5/Cg/theta_prim/"
    list_data = [f for f in os.listdir(path_to_data) if f.endswith('.nc')]

    num_processes = 32

    with Pool(num_processes) as pool:
        list(tqdm(pool.imap(w_prim, list_data), total=len(list_data)))


  0%|          | 0/88 [00:00<?, ?it/s]

19865.nc19851.nc19804.nc19840.nc19809.nc19813.nc19814.nc19792.nc19815.nc19839.nc19843.nc19812.nc19799.nc19837.nc

19794.nc19841.nc19862.nc19877.nc
19824.nc

19826.nc19850.nc

19800.nc19873.nc19834.nc
19795.nc19831.nc
19844.nc




19805.nc19790.nc
19866.nc














19846.nc
19855.nc
19832.nc
19860.nc
19854.nc
19797.nc
19821.nc
19829.nc
19847.nc
19864.nc
19859.nc
19878.nc
19823.nc
19845.nc
19853.nc
19874.nc
19791.nc
19811.nc


  0%|          | 0/88 [03:18<?, ?it/s]


NameError: name 'lats_NH' is not defined

In [ ]:
import numpy as np
from netCDF4 import Dataset
import os
import re
import multiprocessing
import pandas as pd

path_inputs = '/depot/wanglei/data/Reanalysis/ERA5/Cg/theta_prim/'
path_outputs = '/scratch/negishi/yan481/KenYan_Traffic_Model/Valentina Cg Calculation/'

a = 6371000  # Earth radius in meters

def calculate_wrapped_derivative(prev_val, curr_val, next_val, delta):
    diff = next_val - prev_val

    if diff > np.pi:
        return (prev_val + 2 * np.pi - next_val) / (2 * delta)
    elif diff < -np.pi:
        return (prev_val - 2 * np.pi - next_val) / (2 * delta)
    else:
        return (prev_val - next_val) / (2 * delta)

def k_prim(ncfile_i):
    print(ncfile_i)
    theta_prim = np.array(Dataset(f'{path_inputs}{ncfile_i}.nc')['theta_prim'])
    time_i = np.array(Dataset(f'{path_inputs}{ncfile_i}.nc')['time'])
    k_prim = np.zeros_like(theta_prim) * np.nan

    delta_x = 2 * np.pi / 180  # 2 degrees in radians

    for num_lat in range(len(lats_NH)):
        for j in range(1, theta_prim.shape[1]-1):
            for k in range(1, theta_prim.shape[2]-1):
                k_prim[num_lat, j, k] = -calculate_wrapped_derivative(
                    theta_prim[num_lat, j-1, k], 
                    theta_prim[num_lat, j, k], 
                    theta_prim[num_lat, j+1, k], 
                    delta_x
                ) / (a * np.cos(np.radians(lats_NH[num_lat])))

    k_prim[k_prim==0] = np.nan
    save_nc_3d(f'{path_outputs}temporal/k_prim/{str(ncfile_i)}.nc', k_prim, lats_NH, lons, time_i, 'k_prim')

# I'll make a slight adjustment for fetching the list of data
list_data = [f.split('.nc')[0] for f in os.listdir(path_inputs) if f.endswith('.nc')]
list_data = np.sort(np.array([re.findall(r'\d+', file)[0] for file in list_data]).astype(int))

for sublist_data in [sublist.tolist() for sublist in np.array_split(list_data, 12)]:
    if __name__ == "__main__":
        with multiprocessing.Pool(processes=32) as pool:
            pool.map(k_prim, sublist_data)
